In [1]:
import tensorflow as tf
import keras

In [2]:
keras.__version__

'3.11.3'

In [3]:
%pwd

'c:\\Users\\kisho\\Desktop\\SignLanguagePrediction\\SignLanguage_Translation\\research'

In [4]:
%cd ../

c:\Users\kisho\Desktop\SignLanguagePrediction\SignLanguage_Translation


c:\Users\kisho\anaconda3\envs\signlanguage\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from keras.applications import MobileNetV2
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras import layers
from keras.models import Sequential, Model

In [6]:
import json

with open("asl_labels.json") as f:
    labels = json.load(f)
num_classes = len(labels)

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.05, 0.05),
], name="data_augmentation")

inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
conv_base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
conv_base.trainable = True
for layer in conv_base.layers[:-30]:
    layer.trainable = False
x = conv_base(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax", dtype="float32")(x)

model = Model(inputs, outputs)

model.load_weights("model.h5")

In [8]:
import numpy as np
import cv2

# Load and preprocess the image
img = cv2.imread("captured_hand.jpg")  # Replace with your image filename
img = cv2.resize(img, (224, 224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img / 255.0  # Normalize to [0, 1]
img = np.expand_dims(img, axis=0)  # Add batch dimension

pred = model.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


In [10]:
np.argmax(pred)

24

In [12]:
labels['24']

'Y'